# Mesogens with NP | Equilibrium

## Temperature: 6.0 | CPU Run

### Date: 11/14/2019 | System P = 1.8, Expected value of $T_c$ : 7.09 

In [1]:
# -----Import essential packages

from __future__ import division
import hoomd
import hoomd.md

In [2]:
#-----Define important variables

temp = 6.0;
pressure = 1.8;
steps = 1e5;

In [3]:
#-----Declare the file from wich we'll extract the configuration of the system.

ramp_file = "T_CM&NP_" + str(temp) + "_P_" + str(pressure) + "_ramp.gsd"

In [4]:
#-----Start from a gsd file

hoomd.context.initialize("--mode=cpu", memory_traceback = True);

#-----Select the trayectory of the center of mass of particles

system = hoomd.init.read_gsd(ramp_file,frame = -1)

#-----Define each rigid body type in the local coordinate system of the body.

rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
                types=['A']*8,
                positions=[(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                           (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

#-----Instruct the rigid constraint to create the constituent particles

rigid.create_bodies();

#-----Define the potential energy

nl = hoomd.md.nlist.tree(r_buff = 0.5, check_period = 1); #-----Implementation of fast neighborhood list.

lj = hoomd.md.pair.lj(r_cut = 3.5, nlist=nl)
lj.set_params(mode='shift')

#-----Define interaction with species in the binary mixture

lj.pair_coeff.set('NP','NP', epsilon = 1.0, sigma = 5.0);
lj.pair_coeff.set('M' ,'M' , epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('A' ,'A' , epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('M' ,'A' , epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('NP','M' , epsilon = 1.0, sigma = 3.0);
lj.pair_coeff.set('NP','A' , epsilon = 1.0, sigma = 3.0);

#-----Select an standard integrator.

hoomd.md.integrate.mode_standard(dt = 0.005);

#------Define some groups and make their union.

nanoparticles = hoomd.group.type(name = 'Nano_Particles', type = 'NP');
mesogens = hoomd.group.rigid_center();
groupNP_mes = hoomd.group.union(name = "NP_Mes", a = nanoparticles, b = mesogens);

#----Integrate using NPT

npt = hoomd.md.integrate.npt(group = mesogens, kT = temp, tau = 10.0, tauP = 10.0, P = pressure);

#-----Write output and Run the Simulation

log_file = "T_" + str(temp) + "_P_" + str(pressure) + "_equilibrium.log"
gsd_file = "T_" + str(temp) + "_P_" + str(pressure) + "_equilibrium.gsd" 
meso_gsd_file = "T_CM_" + str(temp) + "_P_" + str(pressure) + "_equilibrium.gsd"
log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite=True); 
meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e3,
               group = mesogens,
               overwrite=True); 

HOOMD-blue 2.6.0 DOUBLE HPMC_MIXED TBB SSE SSE2 SSE3 
Compiled: 05/29/2019
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
HOOMD-blue is running on the CPU
notice(2): Group "all" created containing 1004 particles
notice(2): constrain.rigid(): Creating 1000 rigid bodies (adding 8000 particles)
-----
You are using tree neighbor lists. Please cite the following:
* M P Howard, J A Anderson, A Nikoubashman, S C Glotzer, and A Z
  Panagiotopoulos. "Efficient neighbor list calculation for mol

In [5]:
#-----Run the simulation.

hoomd.run(steps)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 9004
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: yes
** starting run **
Time 00:00:10 | Step 443164 / 543000 | TPS 16.3646 | ETA 01:41:40
Time 00:00:20 | Step 443344 / 543000 | TPS 17.9707 | ETA 01:32:25
Time 00:00:30 | Step 443513 / 543000 | TPS 16.8002 | ETA 01:38:41
Time 00:00:40 | Step 443685 / 543000 | TPS 17.1127 | ETA 01:36:43
Time 00:00:50 | Step 443864 / 543000 | TPS 17.8372 | ETA 01:32:37
Time 00:01:00 | Step 444046 / 543000 | TPS 18.0179 | ETA 01:31:31
Time 00:01:10 | Step 444219 / 543000 | TPS 17.1827 | ETA 01:35:48
Time 00:01:20 | Step 444390 / 543000 | TPS 17.0979 | ETA 01:36:07
Time 00:01:30 | Step 444567 / 543000 | TPS 17.3722 | ETA 01:34:26
Time 00:01:40 | Step 444746 / 543000 | TPS 17.7591 | ETA 01:32:12
Time 00:01:50 | Step 444922 / 543000 | TPS 17.4752 | ETA 01:33:32
Time 00:02:00 | Step 445097 / 5

In [6]:
#-----Get density.

system.get_metadata()

OrderedDict([('box', <hoomd.data.boxdim at 0x1159c5d68>),
             ('particles', <hoomd.data.particle_data at 0x115a897b8>),
             ('number_density', 0.6252660196478312),
             ('bonds', <hoomd.data.bond_data at 0x115a89828>),
             ('angles', <hoomd.data.angle_data at 0x115a89898>),
             ('dihedrals', <hoomd.data.dihedral_data at 0x115a89908>),
             ('impropers', <hoomd.data.dihedral_data at 0x115a89978>),
             ('constraints', <hoomd.data.constraint_data at 0x115a899e8>),
             ('pairs', <hoomd.data.bond_data at 0x115a89a58>),
             ('timestep', 543000)])

In [7]:
#-----Get volume.

system.box.get_volume()

14400.270792056355